In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import time
import seaborn as sns
from matplotlib import pyplot as plt
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
import os
from dotenv import load_dotenv
%load_ext sql
import prediction

### Read data and select columns

In [ ]:
orders = pd.read_pickle("../data/battery_cleaned_with_geo.pkl")
orders.columns, orders.shape # redacted

In [3]:
# we work with a copy. at the end of the notebook we will impute predictions to the original data
orders_orig = orders.copy()
orders_orig.reset_index(inplace=True, drop=True)

In [4]:
orders = orders[['übergabestellennummer', 'konzernnummer', 'name_1', 'name_2', 'ort', 'branche', 'typ', 'kreisgemeindeschlüssel', 'bundesland', 'auftragsnummer', 'vertragsnummer', 'transporteur', 'auftragstyp', 'auftragsstatus', 'auftragsdatum',
                  'auftrag_bestätigt_am', 'abholdatum', 'zurückgemeldet_am', 'bruttogewicht_in_kg', 'nettogewicht_in_kg', 'angemeldete_containeranzahl', 'angeforderter_behältertyp', 'gelieferte_behälteranzahl', 'gelieferter_behältertyp', 'volle_Addresse', 'lat', 'long']]

In [ ]:
orders.reset_index(inplace=True, drop=True)
orders.head(5) # redacted

### Plausibility checks

In [ ]:
orders.query("übergabestellennummer == '103509220'") # redacted

In [7]:
q_completed_orders = "abholdatum != '1999-01-01' and zurückgemeldet_am != '1999-01-01'"
q_check_same_container_type = "angeforderter_behältertyp != gelieferter_behältertyp"
q_check_same_group_as_collection_point = "übergabestellennummer == konzernnummer"
q_check_no_weight_but_not_open = "nettogewicht_in_kg == 0 and auftragstyp == 'Abholauftrag' and (auftragsstatus == 'Erledigt' or auftragsstatus == 'Drittbeauftragt')"
q_check_new_carrier_completed = "nettogewicht_in_kg != 0 and auftragsstatus == 'Drittbeauftragt'"


In [8]:
orders.query(q_check_new_carrier_completed).shape[0]

0

In [9]:
orders.query(q_check_no_weight_but_not_open).shape[0]

435

In [10]:
orders.query(q_check_same_group_as_collection_point).shape[0]

42479

In [11]:
orders.query(q_completed_orders).query(q_check_same_container_type).shape[0]

372

In [ ]:
orders.query(q_completed_orders).query(q_check_same_container_type).head(3)
# redacted


picked up containers and delivered empty containers are not always the same type!

This happens for example when the collection point switches from a smaller container type to a bigger one.
This means: for the next pick up we have to calculate the target net_weight for bigger containers.

### Main logic

#### prep code

##### prepare dataframe for later calcualations

In [13]:
def filter_for_report(orders_, debug=True):

    # filter certain collection points based on stakeholders insights: 103508246
    q_übergabestellenummern = "übergabestellennummer == 103508246"
    print(f"drop q_übergabestellenummern rows: {orders_.query(q_übergabestellenummern).shape[0]}")
    orders_ = orders_.drop(orders_.query(q_übergabestellenummern).index)
    return orders_ 

In [14]:
orders = filter_for_report(orders)

drop q_übergabestellenummern rows: 116


In [ ]:
orders_comp, orders_open = prediction.filter_dataframe_for_prediction(orders)
# redacted

In [16]:
# remove orders having unknown container weights
use_pdf_weight = True

if use_pdf_weight:
    weight_dict = prediction.container_net_weights_full_pdf
else:
    weight_dict = prediction.container_net_weights_full_data

drop_list = list(prediction.get_unknown_container_weights_dict(weight_dict).keys())
print(f"unknown container weights: {drop_list}")
orders_comp = orders_comp[orders_comp.gelieferter_behältertyp.isin(drop_list) == False]
orders_comp = orders_comp[orders_comp.angemeldete_containeranzahl.isin(drop_list) == False]
orders_comp.shape

unknown container weights: ['Industriepalette', 'Nicht verpackt', 'Klebeband', 'PE-Beutel a 50 Stück', 'Paloxe', 'Palette PV Industriebatterien', 'Gitterbox', 'Paloxe PV Industriebatterien']


(101258, 27)

In [ ]:
print(orders_comp.query(q_completed_orders).shape[0])
# ok so we have 3 orders not reported back but having a pick-up date...let's ignore them
#assert orders_comp.query(q_completed_orders).shape[0] == 0

In [ ]:
# sorting by company_group, collection_point and order_date ascending
orders_completed_by_company_colpoint_coldate_desc = orders_comp.sort_values(by=['konzernnummer','übergabestellennummer', 'abholdatum'], ascending=True)
orders_completed_by_company_colpoint_coldate_desc.shape

##### calculate mean weights of container-types from data (values go to configuration above)

In [19]:
orders_completed_by_company_colpoint_coldate_desc.gelieferter_behältertyp.unique()

array(['Kiste', 'Fass (60 Ltr.)', 'Fass (Hochenergiebatterien)',
       'Fass Li-Batt gemäß SV376 ADR', '5KG',
       'SET(Fass/Verp./Sleeve rot)', 'Fass (E-Bike)',
       'Sicherheitsbehälter', 'KIT(Verp. / Sleeve rot)'], dtype=object)

In [20]:
orders_completed_by_company_colpoint_coldate_desc["gewicht_behälter_data"] = orders_completed_by_company_colpoint_coldate_desc["bruttogewicht_in_kg"] - orders_completed_by_company_colpoint_coldate_desc["nettogewicht_in_kg"]

In [ ]:
# calculating the mean empty weight of container types
orders_completed_by_company_colpoint_coldate_desc.query("auftragstyp != 'Erstausstattung'").groupby("angeforderter_behältertyp")["gewicht_behälter_data"].mean()

In [ ]:
# check what's up with the 5KG container :)
orders_completed_by_company_colpoint_coldate_desc.query("angeforderter_behältertyp == '5KG' and auftragstyp != 'Erstausstattung'")
# redacted

the 5KG container has no weight since it's only one observation having net and gross weight of 0. This is ok since it will not influence predictions.

#### calculate the day when a collection point's containers are full

**Dynamic dataframe creation**

In SQL we would create one table for each collection point. 
My best idea how to do this in python is to create dynamic variables containing the collection point number in the variable name.
Then we assign a dataframe containing only the rows for this collection point to it.

Example:

In [23]:

""" 
Dynamic_Variable_Name = "orders_collection_point_1672201"
df = orders_completed_by_company_colpoint_coldate_desc.query("übergabestellennummer == 1672201")
 
# The df is assigned to "orders_collection_point_1672201" variable
globals()[Dynamic_Variable_Name] = df
 
# Display variable
orders_collection_point_1672201.head() 
"""


' \nDynamic_Variable_Name = "orders_collection_point_1672201"\ndf = orders_completed_by_company_colpoint_coldate_desc.query("übergabestellennummer == 1672201")\n \n# The df is assigned to "orders_collection_point_1672201" variable\nglobals()[Dynamic_Variable_Name] = df\n \n# Display variable\norders_collection_point_1672201.head() \n'

### Create a list of dataframes for each collection point

In [ ]:
# create a list of all collection_point numbers
 
data = orders_completed_by_company_colpoint_coldate_desc
    
collection_points = list(data.übergabestellennummer.unique())
len(collection_points)

In [ ]:

collection_points.sort()
# create a list for the variable names
collection_point_df_name_list = list()

for cp in collection_points:
    
    dataframe_name = "orders_collection_point_" + cp.astype(str)
    collection_point_df_name_list.append(dataframe_name)
    # The df is assigned to "orders_collection_point_<some-collection-point-id>" variable
    Dynamic_Variable_Name = dataframe_name
    globals()[Dynamic_Variable_Name] = data.query("übergabestellennummer == @cp")

# show the first df in the list
#collection_point_df_name_list[0]
globals()[collection_point_df_name_list[0]]

# redacted

In [26]:
def plot_pick_up_orders_for_collection_point(df, save=False):
    ampel_colors = {"Erledigt" : "grey", "Berechnet": "yellow"}
    plt.figure(figsize=(10,8))
    ax = sns.barplot(data=df, x="abholdatum", y="nettogewicht_in_kg", hue="auftragsstatus", palette=ampel_colors, linewidth=1, edgecolor=".5")
    #ax.set_xticklabels(labels = ['<= 10 days', '> 17 days', '11-15 days', '15-17 days'])
    ax.axhline(df.nettogewicht_in_kg.mean(), color='grey', label='durchschnittliche Abholmenge', linestyle='--')
    ax.set_ylabel("Nettogewicht Abholung in kg")
    ax.set_xlabel("Tag der Abholung")
    ax.set_title("Sammelstelle " + df["übergabestellennummer"].iloc[0].astype(str) + ":  Erledigte und Erwartete Aufträge")
    xticks = ax.get_xticks()
    plt.xticks(rotation=45)
    #ax.set_xticklabels([pd.to_datetime(tm, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for tm in xticks], rotation=45)
    for i in ax.containers:
        ax.bar_label(i, padding=5)
    if(save):
        path = "../images/generated/"
        filename = "prediction_barplot_collection_point_" + df["übergabestellennummer"].iloc[0].astype(str) + ".png"
        plt.savefig(path + filename)

In [27]:
# plot_pick_up_orders_for_collection_point(df_one)

### Automated predictions for all collection_points (no plots). Write predictions into orders_orig.

In [30]:
debug = False # setting this to False will generate predictions for all collection points!

number_collection_points = 3
count = 0
predicted_collection_points_list = list()
for df_name in collection_point_df_name_list:
   if debug:
      if count >= number_collection_points:
         break
   if(debug):
      print(f"df_name: {df_name}")
   df = globals()[df_name]
   # filter out collection points having only one order since we can not calculate a time range between collections for them (total_days_of_collection_all_orders can not be calculated)
   df = prediction.drop_single_pick_ups_and_single_initial_deliveries(df)
   if df.shape[0] > 0:
      pred_dict = prediction.predict_capacity_of_collection_point_full_date(df, prediction.container_net_weights_full_pdf, debug=False)
      pred_date = pred_dict["kapa_erreicht_am"]
      pred_capacity = pred_dict["kapa_kg"]
      pred_row = {'konzernnummer': df["konzernnummer"].iloc[0], 'vertragsnummer': df["vertragsnummer"].iloc[0], 'typ': df["typ"].iloc[0], 'übergabestellennummer': df["übergabestellennummer"].iloc[0], 'auftragstyp': 'Abholauftrag', 'auftragsstatus' : 'Berechnet', 'abholdatum' : pred_date, 'nettogewicht_in_kg': pred_capacity}
      predicted_collection_points_list.append(df["übergabestellennummer"].iloc[0])
      #df = df.append(pred_row, ignore_index = True)
      if(debug):
         print(pred_row)
      #orders_orig = orders_orig.append(pred_row, ignore_index = True)
      orders_comp = orders_comp.append(pred_row, ignore_index = True)
      count += 1

In [ ]:
print(f"predicted {count} dates on which a collection points capacity of kg batteries collected will be reached.")
# redacted

In [ ]:
orders_open.übergabestellennummer.unique().shape[0], orders_comp.übergabestellennummer.unique().shape[0], len(set(predicted_collection_points_list)),
# redacted

In [33]:
# combine open, completed and predicted orders into one df
# use only collection points that enough data to calculate predictions

db_data_open = orders_open[orders_open["übergabestellennummer"].isin(predicted_collection_points_list)]
db_data_comp = orders_comp[orders_comp["übergabestellennummer"].isin(predicted_collection_points_list)]
concat_list = [db_data_open, db_data_comp]
db_data = pd.concat(concat_list)


In [ ]:
db_data.shape, db_data.übergabestellennummer.unique().shape[0]

# redacted

In [ ]:
db_data # redacted

### Store results into DB

In [36]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [37]:
load_dotenv()
DATABASE = os.getenv('DATABASE')
USER_DB = os.getenv('USER_DB')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')

# connect to existing db
con_string = "mysql+pymysql://" + USER_DB + ":" + PASSWORD + "@" + HOST + "/" + DATABASE + "?charset=utf8mb4"
db = create_engine(con_string)
# test connection
pd.read_sql("show databases", db)

,Database
0,battery
1,information_schema
2,mysql
3,performance_schema
4,sys


In [38]:
TABLE_NAME = "ORDERS_PREDICTIONS_ONLY_PREDICTED_CPS_V5"
OVERWRITE_TABLE = False

In [39]:
# write data to table
if OVERWRITE_TABLE:
    pd.read_sql("drop table " + TABLE_NAME, db)
    db_data.to_sql(TABLE_NAME, db)

In [ ]:
# test: select some predictions
df_test = pd.read_sql("Select übergabestellennummer, typ, auftragsnummer, auftragsstatus, abholdatum, nettogewicht_in_kg from " + TABLE_NAME + " WHERE auftragsstatus='Berechnet' limit 10", db)
df_test # redacted

### Visualization Example

In [ ]:
# plot barchart for the initially created dummy table.
df_reporting = pd.read_csv("../data/first_chart.csv")
df_reporting # redacted

In [ ]:
ampel_colors = {"completed" : "green", "predicted": "yellow"}
plt.figure(figsize=(10,8))
ax = sns.barplot(data=df_reporting, x="abholdatum", y="nettogewicht", hue="abholung_typ", palette=ampel_colors, linewidth=1, edgecolor=".5")
#ax.set_xticklabels(labels = ['<= 10 days', '> 17 days', '11-15 days', '15-17 days'])
ax.set_ylabel("Nettogewicht Abholung in kg")
ax.set_xlabel("Tag der Abholung")
ax.set_title("Erledigte und Erwartete Aufträge")
for i in ax.containers:
    ax.bar_label(i, padding=5)

# redacted